In [87]:
from collections import Counter
import numpy as np
import os
# Montar Google Drive
#drive.mount('/content/drive')

 ## Crear archivo input

In [88]:
# Crear un archivo de texto llamado "input.txt"

contenido = "Oíd mortales el grito sagrado libertad libertad libertad oíd el ruido de rotas cadenas ved en trono a la noble igualdad se levanta a la faz de la tierra una nueva y gloriosa nación coronada su sien de laureles y a su planta rendido un león sean eternos los laureles que supimos conseguir coronados de gloria vivamos o juremos con gloria morir ved el trono a la noble igualdad ya su trono dignísimo abrieron las provincias unidas del sur y los libres del mundo responden al gran pueblo argentino salud al gran pueblo argentino salud y los libres del mundo responden al gran pueblo argentino salud"

with open("input.txt", "w", encoding="utf-8") as archivo:
    archivo.write(contenido)

print("Archivo input creado")

Archivo input creado


# Funciones

In [89]:

#===================[probabilidades de ocurrencia]==========================
def analizar_probabilidades(path_archivo):
    """
    Lee un archivo de texto y devuelve un diccionario con la probabilidad
    de aparición de cada carácter en el archivo.

    Parámetros:
        path_archivo (str): Ruta al archivo de texto a analizar.

    Retorna:
        dict: Diccionario con caracteres como claves y sus probabilidades como valores.
    """

    if not os.path.isfile(path_archivo):
        raise FileNotFoundError(f"Archivo no encontrado: {path_archivo}")

    with open(path_archivo, 'r', encoding='utf-8') as archivo:
        texto = archivo.read()

    total_caracteres = len(texto)
    if total_caracteres == 0:
        return {}

    conteo = Counter(texto)

    probabilidades = {caracter: conteo[caracter] / total_caracteres for caracter in conteo}

    return probabilidades


#========================[Entropia]===============================

def calcular_entropia(probabilidades):
    """
    Calcula la entropía del texto, a partir de las probabilidades de los caracteres.

    Parámetros:
        probabilidades (dict): Diccionario con probabilidades de cada carácter.

    Retorna:
        float: Valor de entropía en bits por símbolo.
    """
    if not probabilidades:
        return 0.0

    p = np.array(list(probabilidades.values()))

    # Evitar log2(0) usando enmascarado
    p = p[p > 0]  # Filtra solo valores > 0
    entropia = -np.sum(p * np.log2(p))

    return entropia



#=======================[codificacion Huffman]===============================



def construir_arbol_huffman(probabilidades):
    """
    Construye el árbol de Huffman usando listas ordenadas.

    Retorna:
        nodo raíz del árbol (tupla)
    """
    nodos = [(p, c) for c, p in probabilidades.items()]
    nodos.sort(key=lambda x: x[0])  # Orden inicial por probabilidad

    while len(nodos) > 1:
        # Tomamos los dos nodos con menor probabilidad
        p1, c1 = nodos.pop(0)
        p2, c2 = nodos.pop(0)

        # Fusionamos en un nuevo nodo
        nuevo_nodo = (p1 + p2, [c1, c2])

        # Insertamos manteniendo la lista ordenada por probabilidad
        nodos.append(nuevo_nodo)
        nodos.sort(key=lambda x: x[0])

    return nodos[0][1]  # Retorna solo la estructura del árbol (sin la probabilidad total)


def generar_codigos_huffman(arbol, codigo_actual="", codigos=None):
    """
    Recorre el árbol para asignar códigos binarios a cada carácter.

    Parámetros:
        arbol: estructura del árbol (puede ser carácter o lista [izq, der])
        codigo_actual: cadena binaria recorrida hasta el nodo actual
        codigos: diccionario de salida

    Retorna:
        dict: {carácter: código_binario}
    """
    if codigos is None:
        codigos = {}

    if isinstance(arbol, str):  # es una hoja
        codigos[arbol] = codigo_actual
    else:
        generar_codigos_huffman(arbol[0], codigo_actual + "0", codigos)
        generar_codigos_huffman(arbol[1], codigo_actual + "1", codigos)

    return codigos


def codificar_huffman(probabilidades):
    """
    Implementa Huffman.

    Parámetros:
        probabilidades (dict): {carácter: probabilidad}

    Retorna:
        dict: {carácter: código_binario}
    """
    arbol = construir_arbol_huffman(probabilidades)
    codigos = generar_codigos_huffman(arbol)
    return codigos


def huffmanenco(texto, codigos):
    """
    Codifica el texto original usando el código de Huffman.

    Parámetros:
        texto (str): Texto original.
        codigos (dict): Diccionario {carácter: código_binario}

    Retorna:
        list: Lista de cadenas binarias correspondientes a cada carácter.
    """
    codificado = [codigos[char] for char in texto if char in codigos]
    return codificado


def huffmandeco(mensaje_binario, codigos):
    """
    Decodifica un mensaje binario usando el diccionario de códigos de Huffman.

    Parámetros:
        mensaje_binario (str): Cadena de bits completa.
        codigos (dict): Diccionario {carácter: código_binario}

    Retorna:
        list: Lista de caracteres decodificados.
    """
    # Invertimos el diccionario para buscar código -> carácter
    codigos_invertidos = {v: k for k, v in codigos.items()}

    texto_decodificado = []
    buffer = ""

    for bit in mensaje_binario:
        buffer += bit
        if buffer in codigos_invertidos:
            texto_decodificado.append(codigos_invertidos[buffer])
            buffer = ""  # Reiniciar para buscar el siguiente código

    return texto_decodificado

#====================[Calculo de longitudes]=======================

def calcular_longitudes_codigo(probabilidades, codigos):
    """
    Calcula la longitud mínima y promedio del código de Huffman.

    Parámetros:
        probabilidades (dict): {carácter: probabilidad}
        codigos (dict): {carácter: código binario}

    Retorna:
        tuple: (longitud_minima, longitud_promedio)
    """
    longitudes = {c: len(codigos[c]) for c in codigos}

    # Longitud mínima
    longitud_minima = min(longitudes.values())

    # Longitud promedio (esperada)
    p = np.array([probabilidades[c] for c in codigos])
    l = np.array([longitudes[c] for c in codigos])
    longitud_promedio = np.sum(p * l)

    return longitud_minima, longitud_promedio


#========[Guardar archivo de texto]=======
def guardar_texto_salida(nombre_archivo, caracteres):
    """
    Guarda una lista de caracteres en un archivo de texto.

    Parámetros:
        nombre_archivo (str): Nombre del archivo a guardar.
        caracteres (list): Lista de caracteres (texto decodificado).
    """
    with open(nombre_archivo, 'w', encoding='utf-8') as f:
        f.write(''.join(caracteres))


# Codificacion de fuente

In [90]:
import os
print(os.getcwd())         # debería dar '/content'
print(os.listdir())        # lista archivos en /content

c:\Users\mogic\Git_repos\Taller-de-coms
['.git', 'codes.csv', 'Informe.docx', 'input.txt', 'output.txt', 'TP1.ipynb']


In [91]:
# calculo de probabilidades de ocurrencia
ruta = "input.txt"          # mismo directorio que el .ipynb

probabilidades = analizar_probabilidades(ruta)
print(probabilidades)

{'O': 0.0016806722689075631, 'í': 0.005042016806722689, 'd': 0.05714285714285714, ' ': 0.17647058823529413, 'm': 0.013445378151260505, 'o': 0.08067226890756303, 'r': 0.06722689075630252, 't': 0.02689075630252101, 'a': 0.0957983193277311, 'l': 0.07226890756302522, 'e': 0.08403361344537816, 's': 0.058823529411764705, 'g': 0.025210084033613446, 'i': 0.04873949579831933, 'b': 0.018487394957983194, 'u': 0.04201680672268908, 'c': 0.011764705882352941, 'n': 0.07058823529411765, 'v': 0.011764705882352941, 'f': 0.0016806722689075631, 'z': 0.0016806722689075631, 'y': 0.008403361344537815, 'ó': 0.0033613445378151263, 'p': 0.013445378151260505, 'q': 0.0016806722689075631, 'j': 0.0016806722689075631}


In [92]:
H = calcular_entropia(probabilidades)
print(f"Entropía: {H:.4f} bits/símbolo")

Entropía: 3.9458 bits/símbolo


In [93]:
codigos = codificar_huffman(probabilidades)

# arma un csv de la tabla
ruta_csv = "codes.csv"

with open(ruta_csv, 'w', encoding='utf-8') as archivo:
    archivo.write("Simbolo,Probabilidad,Codigo\n")  # cabecera del CSV
    for caracter in probabilidades:
        prob = probabilidades[caracter]
        codigo = codigos.get(caracter, "")
        archivo.write(f"{caracter},{prob:.4f},{codigo}\n")

In [94]:

l_min, l_prom = calcular_longitudes_codigo(probabilidades, codigos)

print(f"Longitud mínima del código: {l_min} bits")
print(f"Longitud promedio del código: {l_prom:.4f} bits/símbolo")

Longitud mínima del código: 3 bits
Longitud promedio del código: 3.9916 bits/símbolo


In [95]:

with open(ruta, 'r', encoding='utf-8') as archivo:
    texto = archivo.read()


texto_codificado = huffmanenco(texto, codigos)
print (codigos)
mensaje_binario = ''.join(texto_codificado)

print("Primeros 100 bits del mensaje codificado:")
print(mensaje_binario[:100])

{'e': '000', 'a': '001', 'i': '0100', 'g': '01010', 'v': '010110', 'm': '010111', 't': '01100', 'p': '011010', 'O': '011011000', 'f': '011011001', 'z': '011011010', 'q': '011011011', 'y': '0110111', 'd': '0111', 's': '1000', 'r': '1001', 'n': '1010', 'l': '1011', 'o': '1100', 'b': '110100', 'í': '11010100', 'j': '110101010', 'ó': '110101011', 'c': '1101011', 'u': '11011', ' ': '111'}
Primeros 100 bits del mensaje codificado:
0110110001101010001111110101111100100101100001101100010001110001011111010101001010001100110011110000


# Decodificacion de fuente

In [96]:

texto_reconstruido = huffmandeco(mensaje_binario, codigos)

# Convertimos a string si queremos
texto_reconstruido_str = ''.join(texto_reconstruido)

print("Primeras 200 letras decodificadas:")
print(texto_reconstruido_str[:200])


guardar_texto_salida("output.txt", texto_reconstruido)
print("Archivo decodificado guardado como 'output.txt'")


Primeras 200 letras decodificadas:
Oíd mortales el grito sagrado libertad libertad libertad oíd el ruido de rotas cadenas ved en trono a la noble igualdad se levanta a la faz de la tierra una nueva y gloriosa nación coronada su sien de
Archivo decodificado guardado como 'output.txt'
